# Data Download

In [17]:
!wget https://data.mendeley.com/public-files/datasets/fwcj7stb8r/files/fd8717c4-0d65-4c80-a76c-3b61cb68e80d/file_downloaded

--2022-03-19 18:34:28--  https://data.mendeley.com/public-files/datasets/fwcj7stb8r/files/fd8717c4-0d65-4c80-a76c-3b61cb68e80d/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.130.86, 162.159.133.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.130.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/ac918852-7716-4836-ac6f-5755f853b200 [following]
--2022-03-19 18:34:30--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/ac918852-7716-4836-ac6f-5755f853b200
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.120.18
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.120.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179355409 (171M) [applicati

In [18]:
!7za x file_downloaded


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 179355409 bytes (172 MiB)

Extracting archive: file_downloaded
--
Path = file_downloaded
Type = 7z
Physical Size = 179355409
Headers Size = 33229
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 43 - Rice Leaf Disease Images/Bacterialblight/BACTERAILBLIGHT3_039.jpg                                                                             4% 184 - Rice Leaf Disease Images/Bacterialblight/BACTERAILBLIGHT3_180.jpg                                                                   

In [19]:
!rm -r sample_data
!rm file_downloaded
!mv 'Rice Leaf Disease Images' data
!ls

app.py	data  static  templates


# Backend Part

In [20]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 4.4 MB/s 


**Import Libraries**

In [21]:
import tensorflow as tf
import os
from tensorflow.keras import layers, datasets, models
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn import utils

In [22]:
CATEGORIES = ["Bacterialblight" , "Blast" , "Brownspot", "Tungro"]
IMG_SIZE = 200

In [25]:
X = list()
y = list()

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join('/content/data',category)
    category_index = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_arr = cv2.imread(os.path.join(path, img))
      new_arr = cv2.resize(img_arr , (IMG_SIZE, IMG_SIZE))
      X.append(new_arr)
      y.append(category_index)

createTrainingData()
X = np.array(X)
y = np.array(y)   

X, y = utils.shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
cnn = models.Sequential([
  layers.Conv2D(32 ,(3,3), activation='relu' , input_shape=(IMG_SIZE,IMG_SIZE,3)),
  layers.MaxPooling2D((2,2), strides = 2),
  layers.Conv2D(32 , (3,3), activation='relu'),
  layers.MaxPooling2D((2,2), strides = 2),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(1024, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(4 , activation='softmax')
])
cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
cnn.fit(X_train, y_train, epochs=20, batch_size=16)
cnn.evaluate(X_test, y_test)

Epoch 1/20
 72/249 [=======>......................] - ETA: 3:31 - loss: 69.8527 - accuracy: 0.4106

In [ ]:
def predict(filename):
  path = os.path.join('/content/static/upload', filename)
  img = cv2.resize(cv2.imread(path), (IMG_SIZE,IMG_SIZE))
  imgl = [img]
  imgl = np.array(imgl)
  index = cnn.predict(imgl)
  return CATEGORIES[index]

# File Setup for frontend

In [ ]:
!rm -r static templates app.py
!git clone https://github.com/KalWardinX/crop_disease_detection.git
!mv crop_disease_detection/static ./static
!mv crop_disease_detection/templates ./templates
!mv crop_disease_detection/app.py ./app.py
!rm crop_disease_detection

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xf ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 26Xss91r59HCCYKbBsKJsbAhRK7_6QgLMnQQtK4kBhtp6NtMg

# Frontend Part

In [ ]:
!pip install flask_ngrok

**main.py**

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask , render_template,request, flash, redirect, url_for
import os
from werkzeug.utils import secure_filename

In [ ]:
UPLOAD_FOLDER = 'static/upload'
ALLOWED_EXTENSIONS = {'png', 'jpg' ,'jpeg'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.secret_key = 'secret key'
app.config['SESSION_TYPE'] = 'filesystem'


def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/',methods=['GET'])
def index():
    for i in os.listdir(UPLOAD_FOLDER):
        os.remove(UPLOAD_FOLDER+'/'+i)
    return render_template('index.htm', title='crop leaf disease')

@app.route('/',methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        flash("no file part")
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash("no imiage selected for uploading")
        return redirect(request.url)
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        disease = 'nes'#predict(filename)
        return render_template('index.htm', filename=filename, disease=disease)
    else:
        flash("not allowed file")
        return redirect(request.url)

@app.route('/upload/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='upload/'+filename),code=301)

if __name__ == "__main__":
    app.run()
